# Kaggle beolvasás

In [3]:
! pip install -q kaggle

In [4]:
from google.colab import files
files.upload() 

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"markhodszi","key":"1e2bd870a6463ee173677e14f8ae4684"}'}

In [5]:
! mkdir ~/.kaggle

In [6]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [7]:

! kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                      5MB  2022-11-13 15:47:17           2782         81  1.0              
akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                                      9KB  2022-10-06 08:55:25          19162        538  1.0              
thedevastator/jobs-dataset-from-glassdoor                       Salary Prediction                                     3MB  2022-11-16 13:52:31           1970         55  1.0              
fuarresvij/gdp-growth-around-the-globe                      

In [8]:
!kaggle datasets download rashikrahmanpritom/heart-attack-analysis-prediction-dataset -p /content/sample_data/ --unzip

  0% 0.00/4.11k [00:00<?, ?B/s]
100% 4.11k/4.11k [00:00<00:00, 3.02MB/s]


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
data = pd.read_csv("sample_data/heart.csv")
data

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [11]:
from sklearn.model_selection import train_test_split  
lista = data.columns.values.tolist()
lista.remove("output")
x_train, x_test, y_train, y_test= train_test_split(data.loc[:,lista], data.loc[:, "output"], test_size= 0.2, random_state=0)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [13]:
rForest_model = RandomForestClassifier()
rForest_parameters = {'n_estimators':[25, 50, 100, 200, 500, 1000, 5000], 'criterion':('gini', 'entropy'), 'max_features' : ('log2','sqrt')}

In [14]:
svc_model = SVC()
svc_parameters = {'kernel':('linear', 'rbf'), 'C':[1, 5, 10, 20, 50, 100]}

In [15]:
rForest_result = GridSearchCV(rForest_model, rForest_parameters, scoring='recall')
rForest_result.fit(x_train,y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_features': ('log2', 'sqrt'),
                         'n_estimators': [25, 50, 100, 200, 500, 1000, 5000]},
             scoring='recall')

In [16]:
svc_result = GridSearchCV(svc_model, svc_parameters, scoring='recall')
svc_result.fit(x_train,y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 5, 10, 20, 50, 100],
                         'kernel': ('linear', 'rbf')},
             scoring='recall')

In [18]:

print("rForest: ",rForest_result.best_score_)
print("svc: ",svc_result.best_score_)
if rForest_result.best_score_ > svc_result.best_score_:
  model = rForest_result.best_estimator_

else:
  model = svc_result.best_estimator_

rForest:  0.8943019943019943
svc:  0.8786324786324787


In [19]:
model.fit(x_train, y_train)
kimenet = model.predict(x_test)

In [20]:
from sklearn.metrics import recall_score
print(recall_score(y_test, kimenet))

0.8823529411764706


In [21]:
print(model.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [24]:
import pickle

In [25]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [26]:
loaded_model = pickle.load(open(filename,'rb'))

In [32]:
input_data = (30,	1,	0,	200,	300,	0,	1,	123,	1,	0.2,	1,	0,	3)
input_data_ar = np.asarray(input_data)
input_data_re = input_data_ar.reshape(1,-1)

prediction = loaded_model.predict(input_data_re)

if prediction[0] == 0:
    print('The person\'s not gonna have a heartattack')
else:
    print( 'The person\'s gonna have a heartattack')

The person's not gonna have a heartattack


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
